## Resolve dependências

In [1]:
!pip install -q albumentations==1.4.6 opencv-python pycocotools tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pycocotools albumentations==1.4.6 opencv-python tqdm
!pip uninstall -y sympy
!pip install sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 6.3 MB/s eta 0:00:00
Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.4 MB/s eta 0:00:00


In [2]:
import sys
# sys.path.append("/content/drive/MyDrive/TCC/src/")
sys.path.append("/content/drive/MyDrive/TCC/notebooks/Faster R-CNN")

In [3]:
from pathlib import Path

DATASET_ROOT = Path("/content/drive/MyDrive/TCC/DATASET")
paths = {
    "bh_pools": {
        "images": {
            "train": DATASET_ROOT/"bh_pools/images/train",
            "val":   DATASET_ROOT/"bh_pools/images/val",
            "test":  DATASET_ROOT/"bh_pools/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_pools/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_pools/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_pools/annotations/instances_test.json",
        }
    }
}

## Carrega Loaders

In [4]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


pools_train = CocoDetDataset(
    paths["bh_pools"]["images"]["train"],
    paths["bh_pools"]["ann"]["train"],
    transforms=get_transforms(train=True))

pools_val   = CocoDetDataset(
    paths["bh_pools"]["images"]["val"],
    paths["bh_pools"]["ann"]["val"],
    transforms=get_transforms(train=False))

pools_test  = CocoDetDataset(
    paths["bh_pools"]["images"]["test"],
    paths["bh_pools"]["ann"]["test"],
    transforms=get_transforms(train=False))


pools_train_loader = DataLoader(pools_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
pools_val_loader   = DataLoader(pools_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
pools_test_loader  = DataLoader(pools_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

loading annotations into memory...
Done (t=1.85s)
creating index...
index created!
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


## Inicializa model

In [5]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 1 + 2

weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth


100%|██████████| 167M/167M [00:00<00:00, 231MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [6]:
checkpoint_path = '/content/drive/MyDrive/TCC/checkpoints/fasterrcnn_pools_focused.pth'
state = torch.load(checkpoint_path)
model.load_state_dict(state)

<All keys matched successfully>

## Avalia Modelo

In [8]:
from my_utils.my_utils import evaluate_ap_multi_iou
import numpy as np

csv_path = "/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/pool/results.csv"
df_results = evaluate_ap_multi_iou(
    model=model,
    loader=pools_test_loader,
    device=device,
    score_thr=0.25,
    iou_list=(0.50, 0.75, 0.90),
    iou_grid=np.arange(0.50, 0.96, 0.05),
    out_csv_path=csv_path
)

==== Resultados por classe ====
Classe 1 | mAP[.50:.95]=0.490 | AP@0.50=0.787 | AP@0.75=0.542 | AP@0.90=0.047 (GT=1109)

📊 CSV salvo em: /content/drive/MyDrive/TCC/notebooks/Faster R-CNN/pool/results.csv


In [9]:
from my_utils.my_utils import show_predictions

show_predictions(
    model=model,
    loader=pools_test_loader,
    device=device,
    n=2,
    score_thr=0.25
)


Output hidden; open in https://colab.research.google.com to view.